In [ ]:
# -*- coding: utf-8 -*-
"""
Binance Futures Klines Downloader (sliding window)

This script downloads historical futures candlestick (kline) data from Binance.
- Uses Binance Futures REST API endpoint: /fapi/v1/klines
- Supports configurable interval and number of days
- Handles Binance's max limit (1500 rows per request) by sliding window
- Converts timestamps to datetime and numeric fields to float
- Optionally saves the result to CSV

Example:
    $ python download_futures_klines.py
"""

import requests
import pandas as pd
import time
from datetime import datetime

BASE_URL = "https://fapi.binance.com/fapi/v1/klines"


def get_futures_klines(symbol, interval, start, end, limit=1500):
    """
    Fetch a batch of Binance Futures klines between start and end timestamps.

    Parameters
    ----------
    symbol : str
        Trading pair symbol, e.g. "BTCUSDT".
    interval : str
        Kline interval, e.g. "1m", "5m", "1h".
    start : int
        Start time in milliseconds.
    end : int
        End time in milliseconds.
    limit : int, optional
        Max number of rows per request (default=1500).

    Returns
    -------
    pd.DataFrame
        DataFrame of klines with proper column names and dtypes.
        Empty DataFrame if no data returned.
    """
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
        "startTime": start,
        "endTime": end,
    }
    resp = requests.get(BASE_URL, params=params)
    resp.raise_for_status()
    data = resp.json()

    columns = [
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_volume", "trades",
        "taker_base_volume", "taker_quote_volume", "ignore"
    ]
    df = pd.DataFrame(data, columns=columns)
    if df.empty:
        return df

    # Convert datatypes
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
    numeric_cols = [
        "open", "high", "low", "close", "volume",
        "quote_volume", "taker_base_volume", "taker_quote_volume"
    ]
    df[numeric_cols] = df[numeric_cols].astype(float)
    return df


def download_all_klines(symbol, interval="1m", days=30, save_csv=True):
    """
    Download all Binance Futures klines for a given symbol and time range.

    Parameters
    ----------
    symbol : str
        Trading pair symbol, e.g. "BTCUSDT".
    interval : str, optional
        Kline interval (default "1m").
    days : int, optional
        Number of days to download (default 30).
    save_csv : bool, optional
        Whether to save the result to CSV (default True).

    Returns
    -------
    pd.DataFrame
        Concatenated DataFrame of all klines.
        Empty DataFrame if nothing retrieved.
    """
    end_time = int(time.time() * 1000)  # current time (ms)
    start_time = end_time - days * 24 * 60 * 60 * 1000

    all_dfs = []
    # Calculate step size (ms) per API call
    minutes = int(interval.replace("m", "")) if "m" in interval else 1
    step = 1500 * 60 * minutes * 1000
    current = start_time

    while current < end_time:
        next_end = min(current + step, end_time)
        df = get_futures_klines(symbol, interval, current, next_end)
        if not df.empty:
            all_dfs.append(df)
            print(f"✅ Got {len(df)} rows from {df.iloc[0]['open_time']} to {df.iloc[-1]['close_time']}")
        else:
            print("⚠️ Empty response, stopping...")
            break
        current = next_end + 1
        time.sleep(0.3)  # to avoid hitting API rate limits

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        if save_csv:
            filename = f"{symbol}_{interval}_{days}d.csv"
            final_df.to_csv(filename, index=False)
            print(f"💾 Saved to {filename} ({len(final_df)} rows)")
        return final_df
    else:
        return pd.DataFrame()


# ===============================
# Run example
# ===============================
if __name__ == "__main__":
    coin_ls = ['BTC', 'ETH']
    for coin in coin_ls:
        df = download_all_klines(f"{coin}USDT", interval="1m", days=1)
